# C++ Syntax Recipes - A Cheat Sheet 

## Hauptprogramm
```C++
int main(int argc, char *argv[]){
    return 0;
}
```

## Input/Output
Den nötigen Header
```C++
#include <iostream>
```
und die Verwendung:
```C++
auto input = int{}; // Default initialisiert mit 0 
std::cin >> input; 
std::cout << "Das ist der Output der eingelesenen Zahl: " << input;
```

## Variablen *deklarieren* (und *initialisieren*)
Am Besten:
```C++
auto x = wert;
```

Wenn expliziter Typ notwendig (z.B. `vector<int>`):
```C++
auto x = Typ{wert};
```
Die veraltete Form vermeiden:
```C++
Typ x = wert;
```

### Basisdatentypen
```C++
using namespace std::string_literals;
auto an_integer = 4;
auto a_double_value = 123.456;
auto a_string = "Ich bin Text!"s;
auto some_variable = some_function(123.456);
```
So ginge das ohne `auto`. Neuer Code sollte nicht mehr so aussehen:
```C++
using namespace std::string_literals;
int an_integer {4};
double a_double_value {123.456};
std::string a_string {"Ich bin Text!"s};
some_namespace::some_variable_type a_variable {some_function(123.456)};
```

### Container
Erst die zugehörigen header einbinden: Siehe: http://www.cplusplus.com/reference/stl/
```C++
#include <vector>
// Oder
#include <unordered_map>
// Oder
#include <array>
// Oder ...
#include <set>
```
Dann können sie verwendet werden:

```C++
auto an_integer_vector = std::vector<int>{1, 11, 111, 1111};
auto an_int_to_string_mapping = std::unordered_map<int, std::string> {{0, "Typ GRIB"}, {1, "Typ FA"}};
auto a_double_array_of_size_3 = std::array<double, 3>{10.1, 20.2, 30.3};
auto a_set = std::set<int>{1, 5, 1, 1, 2};
```
Und wie man früher ohne `auto` programmiert hat:
```C++
std::vector<int> an_integer_vector {1, 11, 111, 1111};
std::unordered_map<int, std::string> an_int_to_string_mapping {{0, "Typ GRIB"}, {1, "Typ FA"}};
std::array<double, 3> a_double_array_of_size_3 {10.1, 20.2, 30.3};
std::set<int> a_set {1, 5, 1, 1, 2};
```

## Funktionen definieren
```C++
Rueckgabetyp name_der_funktion(Parametertyp1 parameter1, Parametertyp2 parameter2, usw...){
    ...
    return ...;
}
```
`Rueckgabetyp` kann auch `auto` sein. `Parametertyp` muss explizit angegeben werden. (Das könnte man mit templates umgehen.)

Für die Wahl zwischen Kopie und Referenz der Argumente ist folgendes zu empfehlen:
### Standardmäßig by value Kopie verwenden:

Parameter wird kopiert, Wert außerhalb der Funktion bleibt unverändert. Eine Funktion sollte möglichst keine Nebeneffekte haben (nichts außerhalb der Funktion ändern), sondern einen klar definierten Input und Output. Das macht Software testbarer und weniger fehleranfällig.

```C++
double plus_thirteen(double x){
    return x+13.;
}

auto square_entries(std::vector<double> v){
    // We need a copy of v for the return value anyway
    for (auto& entry : v){
        entry *= entry;
    }
    return v;
}
```


### Wenn kopieren zu langsam ist (Bei großen Objekten):
Wenn Kopien teuer sind dann verwende Referenzen auf konstante Variablen mittels `const &`.
Es wird die Variable außerhalb der Funktion verwendet, aber sie bleibt garantiert unverändert. Der Compiler überprüft, dass innerhalb der Funktion kein Schreibzugriff passiert.
```C++
double sum_all(const std::vector<double>& v){
    return accumulate(begin(v), end(v), 0.0);
}
```

##Kontrollstrukturen:
### if-else
```C++
if (condition_holds){
    do_something();
}else{
    do_something_else();
}
```
### Switch
```C++
int type = 2;
switch(type) {
    case 1: std::cout << "FA-File"; 
            break;
    case 2: std::cout << "Grib-File";
            break;
    case 3: std::cout << "ODB-File";
            break;
}
```
**Praxistipp**: Komplizierte `switch` Anweisungen sollte man eher vermeiden. Verwende stattdessen z.B. eine `map` oder `unordered_map`:

```C++
#include <unordered_map>
...
auto file_types = std::unordered_map<int, std::string> {{1, "FA-File"}, {2, "Grib-File"}, {3, "ODB-File"}};
auto type = 2;
std::cout << file_types[type];
```



## Schleifen:
Bevor man beginnt eine Schleife zu schreiben sollte man erst in der Standard-Bibliothek nachsehen ob nicht ein vorgefertigter Algorithmus existiert:
- http://www.cplusplus.com/reference/algorithm/
- http://www.cplusplus.com/reference/numeric/

Wenn es of darum geht Vektoren eintragsweise zu addieren/multiplizieren/etc. dann sollte man statt dem `std::vector` einen numerischen Datentyp verwenden, hier gibt es Bibliotheken wie "eigen" oder "armadillo". Mit deren Hilfe kann man arithmetische Operationen eintragsweise anwenden: `(vec_1+vec_2)*vec_1` ähnlich wie in FORTRAN, numpy oder R.


- http://eigen.tuxfamily.org/
- http://arma.sourceforge.net/

### Zählschleife:
```C++
int max = 10;
for(int i = 0; i<max; ++i){
    std::cout << "Wert von i ist: " << i;
}
```
### Bedingungsschleife:
```C++
while (condition_holds){
    do_something();
}
```

### Schleifen um Container zu ändern:

####Eintragsweise per auto-Referenz einfangen. "Range-based for-loop"
```C++
std::vector<double> v {1.2, 3.4, 5.6, 7.6};
for (auto& e: v){
    e = e*e;
}
```
####Oder `transform` und Funktion benützen:
```C++
using std::begin;
using std::end;
std::vector<double> v {1.2, 3.4, 5.6, 7.6};
double square(double x){
    return x*x;
}
std::transform(begin(v), end(v), begin(v), square);
```
####Oder `transform` und lambda
Ein lambda ist eine Funktion ohne Namen, die man mittels `[](Parametertyp p1,...){...};` erstellt.
Analog zur Funktion `square` von oben kann man auch schreiben:
```C++
using std::begin;
using std::end;
std::vector<double> v {1.2, 3.4, 5.6, 7.6};
std::transform(begin(v), end(v), begin(v), [](double x){return x*x;});
```

#### Eher veraltet: Mittels iterator (bevor die range-based for-loop in C++11 eingeführt wurde)
```C++
std::vector<double> v {1.2, 3.4, 5.6, 7.6};
for (auto it = begin(v); it != end(v); ++it){
    *it = (*it)*(*it);
}
```

#### Nur bedingt empfehlenswert: Zählschleife wie in C
```C++
std::vector<double> v {1.2, 3.4, 5.6, 7.6};
for (auto i = std::vector<double>::size_type{0}; i < v.size(); ++i){
    v[i] = v[i]*v[i]; 
}
```
Nur sinnvoll wenn Index `i` **wirklich** explizit gebraucht wird. (Im obigen Beispiel nicht.)
(Beachte außerdem die Verwendung von `std::vector<double>::size_t` statt `unsigned int`.)


### Schleifen um Container nicht zu ändern:

#### Range-based-for mit `const`
```C++
std::vector<double> v {1.2, 3.4, 5.6, 7.6};
for (const auto& e: v){
    std::cout << e;
}
```

#### Eine Alternative zur ersten Zählschleife - Für jeden Eintrag eine Funktion ausführen:
Die Standardbibliothek enthält auch eine Funktion `for_each`, (siehe: http://www.cplusplus.com/reference/algorithm/ ) Diese kann für jedes Element eine Funktion ausführen.
```C++
using namespace std;
auto v = vector<int>(10);  // Create 10 ints.
iota(begin(v), end(v), 0); // Fill from 0 to 9.
void print_value(int i){
    cout << "Der Wert von i ist" << i << endl;
}
for_each(begin(v), end(v), print_value); 
```

## Pointer
**Wenn möglich vermeiden**. (Es ist im Prinzip immer möglich.) 
```C++
int i = 123;
int* i_ptr = &i; // i_ptr points to address of i
std::cout << "The memory position i_ptr is: " << i_ptr << std::endl;
std::cout << "The value at the memory position i_ptr points to is: " << *i_ptr; // Will output "123"
```
Verwende Referenzen, `unique_ptr`, `shared_ptr` und `weak_ptr` statt Pointern. In 90% der Fälle werden Pointer verwendet um bei der Funktionsübergabe keine Kopien zu erstellen. 
Das ging in C nicht anders. In C++ verwendet man hier stattdessen Referenzen.

Beispiel:
Alter C-Code
```C++
// C CODE!!!
double sum_matrix_entries(Matrix* m){
    ...
}
Matrix* mat_ptr = generate_matrix(1000,1000);
... // Change some matrix entries
double sum = sum_entries(mat_ptr);
delete_matrix(mat_ptr);
```
In C++ würde das so aussehen:
```C++
double sum_entries(const Matrix& m){
    ...
}
Matrix mat(1000,1000);
... // Change some matrix entries
double sum = sum_entries(mat);
```
Beachte dass in C++ kein explizites löschen der Matrix nötig ist! Bei Blockende wird diese automatisch gelöscht! In C gab es oft die Design-Frage: Welche Funktion ist verantwortlich für das freigeben des Speichers? Ein Vergessen des Freigebens führte zu einem Speicherleck. Dieses Problem kann man mit C++ weitestgehend vermeiden.

# Klassen
#TODO TODO TODO


## Arithmetik mit Vektoren
### Verwende zum Beispiel "Eigen" (http://eigen.tuxfamily.org/)
Hierfür muss der Eigen-Header heruntergeladen werden: http://eigen.tuxfamily.org/dox/GettingStarted.html
```C++
#include <iostream>
#include <Eigen/Dense>
using namespace Eigen;
int main()
{
  Matrix2d a;
  a << 1, 2,
       3, 4;
  MatrixXd b(2,2);
  b << 2, 3,
       1, 4;
  std::cout << "a + b =\n" << a + b << std::endl;
  std::cout << "a - b =\n" << a - b << std::endl;
  std::cout << "Doing a += b;" << std::endl;
  a += b;
  std::cout << "Now a =\n" << a << std::endl;
  Vector3d v(1,2,3);
  Vector3d w(1,0,0);
  std::cout << "-v + w - v =\n" << -v + w - v << std::endl;
}
```

## Standard-Algorithmen:
- http://www.cplusplus.com/reference/numeric/
- http://www.cplusplus.com/reference/algorithm

### Numerical operations

- `accumulate`:
    Accumulate values in range (function template )

- `adjacent_difference`:
    Compute adjacent difference of range (function template )

- `inner_product`:
    Compute cumulative inner product of range (function template )

- `partial_sum`:
    Compute partial sums of range (function template )

- `iota`:
    Store increasing sequence (function template )

###Non-modifying sequence operations:

- `all_of`:
    Test condition on all elements in range (function template )

- `any_of`:
    Test if any element in range fulfills condition (function template )

- `none_of`:
    Test if no elements fulfill condition (function template )

- `for_each`:
    Apply function to range (function template )

- `find`:
    Find value in range (function template )

- `find_if`:
    Find element in range (function template )

- `find_if_not`:
    Find element in range (negative condition) (function template )

- `find_end`:
    Find last subsequence in range (function template )

- `find_first_of`:
    Find element from set in range (function template )

- `adjacent_find`:
    Find equal adjacent elements in range (function template )

- `count`:
    Count appearances of value in range (function template )

- `count_if`:
    Return number of elements in range satisfying condition (function template )

- `mismatch`:
    Return first position where two ranges differ (function template )

- `equal`:
    Test whether the elements in two ranges are equal (function template )

- `is_permutation`:
    Test whether range is permutation of another (function template )

- `search`:
    Search range for subsequence (function template )

- `search_n`:
    Search range for elements (function template )


###Modifying sequence operations:

- `copy`:
    Copy range of elements (function template )

- `copy_n`:
    Copy elements (function template )

- `copy_if`:
    Copy certain elements of range (function template )

- `copy_backward`:
    Copy range of elements backward (function template )

- `move`:
    Move range of elements (function template )

- `move_backward`:
    Move range of elements backward (function template )

- `swap`:
    Exchange values of two objects (function template )

- `swap_ranges`:
    Exchange values of two ranges (function template )

- `iter_swap`:
    Exchange values of objects pointed to by two iterators (function template )

- `transform`:
    Transform range (function template )

- `replace`:
    Replace value in range (function template )

- `replace_if`:
    Replace values in range (function template )

- `replace_copy`:
    Copy range replacing value (function template )

- `replace_copy_if`:
    Copy range replacing value (function template )

- `fill`:
    Fill range with value (function template )

- `fill_n`:
    Fill sequence with value (function template )

- `generate`:
    Generate values for range with function (function template )

- `generate_n`:
    Generate values for sequence with function (function template )

- `remove`:
    Remove value from range (function template )

- `remove_if`:
    Remove elements from range (function template )

- `remove_copy`:
    Copy range removing value (function template )

- `remove_copy_if`:
    Copy range removing values (function template )

- `unique`:
    Remove consecutive duplicates in range (function template )

- `unique_copy`:
    Copy range removing duplicates (function template )

- `reverse`:
    Reverse range (function template )

- `reverse_copy`:
    Copy range reversed (function template )

- `rotate`:
    Rotate left the elements in range (function template )

- `rotate_copy`:
    Copy range rotated left (function template )

- `random_shuffle`:
    Randomly rearrange elements in range (function template )

- `shuffle`:
    Randomly rearrange elements in range using generator (function template )


###Partitions:

- `is_partitioned`:
    Test whether range is partitioned (function template )

- `partition`:
    Partition range in two (function template )

- `stable_partition`:
    Partition range in two - stable ordering (function template )

- `partition_copy`:
    Partition range into two (function template )

- `partition_point`:
    Get partition point (function template )


###Sorting:

- `sort`:
    Sort elements in range (function template )

- `stable_sort`:
    Sort elements preserving order of equivalents (function template )

- `partial_sort`:
    Partially sort elements in range (function template )

- `partial_sort_copy`:
    Copy and partially sort range (function template )

- `is_sorted`:
    Check whether range is sorted (function template )

- `is_sorted_until`:
    Find first unsorted element in range (function template )

- `nth_element`:
    Sort element in range (function template )


###Binary search (operating on partitioned/sorted ranges):

- `lower_bound`:
    Return iterator to lower bound (function template )

- `upper_bound`:
    Return iterator to upper bound (function template )

- `equal_range`:
    Get subrange of equal elements (function template )

- `binary_search`:
    Test if value exists in sorted sequence (function template )


###Merge (operating on sorted ranges):

- `merge`:
    Merge sorted ranges (function template )

- `inplace_merge`:
    Merge consecutive sorted ranges (function template )

- `includes`:
    Test whether sorted range includes another sorted range (function template )

- `set_union`:
    Union of two sorted ranges (function template )

- `set_intersection`:
    Intersection of two sorted ranges (function template )

- `set_difference`:
    Difference of two sorted ranges (function template )

- `set_symmetric_difference`:
    Symmetric difference of two sorted ranges (function template )


###Heap:

- `push_heap`:
    Push element into heap range (function template )

- `pop_heap`:
    Pop element from heap range (function template )

- `make_heap`:
    Make heap from range (function template )

- `sort_heap`:
    Sort elements of heap (function template )

- `is_heap`:
    Test if range is heap (function template )

- `is_heap_until`:
    Find first element not in heap order (function template )


###Min/max:

- `min`:
    Return the smallest (function template )

- `max`:
    Return the largest (function template )

- `minmax`:
    Return smallest and largest elements (function template )

- `min_element`:
    Return smallest element in range (function template )

- `max_element`:
    Return largest element in range (function template )

- `minmax_element`:
    Return smallest and largest elements in range (function template )


###Other:

- `lexicographical_compare`:
    Lexicographical less-than comparison (function template )

- `next_permutation`:
    Transform range to next permutation (function template )

- `prev_permutation`:
    Transform range to previous permutation (function template )